In [ ]:
import os

input_path = "/kaggle/input"
for root, dirs, files in os.walk(input_path):
    for file in files:
        if file.endswith(".zip"):
            print(os.path.join(root, file))


In [ ]:
!pip install kagglehub

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("hasyimabdillah/workoutfitness-video")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/workoutfitness-video


In [ ]:
import os

for root, dirs, files in os.walk(extract_path):
    print(f"Directory: {root}")
    for file in files[:5]:  # print first 5 files
        print(f"  {file}")
    break  # avoid printing everything


NameError: name 'extract_path' is not defined

# Install Dependencies

In [ ]:
!pip install transformers datasets evaluate decord --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 64.4 MB/s eta 0:00:00


In [ ]:
!pip install torch

   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/212.5 MB ? eta -:--:--
   ---------------------------------------- 0.8/212.5 MB 2.1 MB/s eta 0:01:41
   ---------------------------------------- 1.6/212.5 MB 2.7 MB/s eta 0:01:18
   ---------------------------------------- 2.6/212.5 MB 3.3 MB/s eta 0:01:04
    --------------------------------------- 3.7/212.5 MB 3.8 MB/s eta 0:00:55
   - -------------------------------------- 5.5/212.5 MB 4.6 MB/s eta 0:00:46
   - -------------------------------------- 7.6/212.5 MB 5.5 MB/s eta 0:00:38
   - -------------------------------------- 10.5/212.5 MB 6.6 MB/s eta 0:00:31
   -- ------------------------------------- 14.2/212.5 MB 7.9 MB/s eta 0:00:25
   --- ------------------------------------ 19.4/212.5 MB 9.6 MB/s eta 0:00:21
   ---- ----------------------------------- 24.9/212.5 MB 11.3 MB/s eta 0:00:17
  

In [ ]:
'''!pip uninstall torch torchvision torchaudio -y
!pip cache purge

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117'''


Found existing installation: torch 2.7.0
Uninstalling torch-2.7.0:
  Successfully uninstalled torch-2.7.0


You can safely remove it manually.


Files removed: 1390
Looking in indexes: https://download.pytorch.org/whl/cu117


ERROR: Could not find a version that satisfies the requirement torch (from versions: none)
ERROR: No matching distribution found for torch


In [ ]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [ ]:
'''pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu'''


Looking in indexes: https://download.pytorch.org/whl/cpu
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------------ --------------------- 0.8/1.7 MB 2.0 MB/s eta 0:00:01
   ------------------------------ --------- 1.3/1.7 MB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 2.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 3.4 MB/s eta 0:00:01
   ---------------- ----------------------- 1.0/2.5 MB 3.1 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 3.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from transformers import VideoMAEImageProcessor
feature_extractor = VideoMAEImageProcessor.from_pretrained("MCG-NJU/videomae-base")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# 1. Set Up Paths and Labels

In [ ]:
import os
from glob import glob

data_dir = "/kaggle/input/workoutfitness-video"
all_classes = sorted(os.listdir(data_dir))

label2id = {label: idx for idx, label in enumerate(all_classes)}
id2label = {idx: label for label, idx in label2id.items()}


# 2: Load and Process Videos into Frames

In [ ]:
!pip install decord

In [ ]:
import decord
from decord import VideoReader
import numpy as np
from torch.utils.data import Dataset
import torch
from transformers import VideoMAEImageProcessor
feature_extractor = VideoMAEImageProcessor.from_pretrained("MCG-NJU/videomae-base")


class ExerciseDataset(Dataset):
    def __init__(self, video_paths, labels, extractor, num_frames=16):
        self.video_paths = video_paths
        self.labels = labels
        self.extractor = extractor
        self.num_frames = num_frames

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        vr = VideoReader(self.video_paths[idx])
        total_frames = len(vr)
        frame_indices = np.linspace(0, total_frames - 1, self.num_frames, dtype=int)
        frames = vr.get_batch(frame_indices).asnumpy()  # shape: (16, H, W, 3)

        inputs = self.extractor(list(frames), return_tensors="pt")
        return {
            "pixel_values": inputs["pixel_values"].squeeze(0),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }


# Build Dataset Lists

In [ ]:
from sklearn.model_selection import train_test_split

video_paths = []
video_labels = []

for label in all_classes:
    paths = glob(os.path.join(data_dir, label, "*.mp4"))
    video_paths.extend(paths)
    video_labels.extend([label2id[label]] * len(paths))

train_paths, val_paths, train_labels, val_labels = train_test_split(
    video_paths, video_labels, test_size=0.2, stratify=video_labels, random_state=42
)

train_dataset = ExerciseDataset(train_paths, train_labels, feature_extractor)
val_dataset = ExerciseDataset(val_paths, val_labels, feature_extractor)


# 4. Load VideoMAE Model

In [ ]:
from transformers import VideoMAEForVideoClassification

model = VideoMAEForVideoClassification.from_pretrained(
    "MCG-NJU/videomae-base",
    num_labels=len(all_classes),
    label2id=label2id,
    id2label=id2label
)


Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 5: Define Training Arguments

In [ ]:
!pip install --upgrade transformers


In [ ]:
import transformers
print(transformers.__version__)


4.52.3


In [ ]:
import transformers
print(transformers.__file__)


/usr/local/lib/python3.11/dist-packages/transformers/__init__.py


In [ ]:
import transformers
print(transformers.__version__)
print(transformers.TrainingArguments.__init__.__code__.co_varnames)

4.52.3
('self', 'output_dir', 'overwrite_output_dir', 'do_train', 'do_eval', 'do_predict', 'eval_strategy', 'prediction_loss_only', 'per_device_train_batch_size', 'per_device_eval_batch_size', 'per_gpu_train_batch_size', 'per_gpu_eval_batch_size', 'gradient_accumulation_steps', 'eval_accumulation_steps', 'eval_delay', 'torch_empty_cache_steps', 'learning_rate', 'weight_decay', 'adam_beta1', 'adam_beta2', 'adam_epsilon', 'max_grad_norm', 'num_train_epochs', 'max_steps', 'lr_scheduler_type', 'lr_scheduler_kwargs', 'warmup_ratio', 'warmup_steps', 'log_level', 'log_level_replica', 'log_on_each_node', 'logging_dir', 'logging_strategy', 'logging_first_step', 'logging_steps', 'logging_nan_inf_filter', 'save_strategy', 'save_steps', 'save_total_limit', 'save_safetensors', 'save_on_each_node', 'save_only_model', 'restore_callback_states_from_checkpoint', 'no_cuda', 'use_cpu', 'use_mps_device', 'seed', 'data_seed', 'jit_mode_eval', 'use_ipex', 'bf16', 'fp16', 'fp16_opt_level', 'half_precision_ba

In [ ]:
!pip install -U accelerate>=0.26.0


In [ ]:
pip install --upgrade accelerate>=0.26.0


Note: you may need to restart the kernel to use updated packages.


In [ ]:
accelerate.__version__


NameError: name 'accelerate' is not defined

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./videomae-finetuned",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    logging_dir='./logs',
    fp16=True,
    report_to="none"
)


# Define Accuracy Metric

In [ ]:
!pip install evaluate

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return accuracy.compute(predictions=preds, references=labels)


# Train the model

In [ ]:
!pip install tf-keras


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.449928,0.220339
2,No log,1.728371,0.508475


In [ ]:
trainer.evaluate()
